## Import using package

In [ ]:
import os
import cv2
import csv
import time
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

from tqdm.notebook import tqdm as tqdm

In [ ]:
train_img_path_lis = glob.glob("../../../0801_Unet_test/0801/train_0801/image/*.PNG")
train_img_name_lis = [os.path.basename(x) for x in train_img_path_lis ]

In [ ]:
# path config 
result_file_path = '../csv_file/PBL_match_result_fold4.csv'

# middle csv config
Resave = True
csv_file_path = '../csv_file/PBL_result_all_cej_bone_fold4.csv'
Dentist_predict_result_csv_path = '../../data/2_ground_truth/HV_stage_GT.csv'
# Dentist label excel

In [ ]:
csv_path1 = '../PBL_result_1.csv'
csv_file1 = pd.read_csv(csv_path1)
csv_path2 = '../PBL_result_2.csv'
csv_file2 = pd.read_csv(csv_path2)
csv_path3 = '../PBL_result_3.csv'
csv_file3 = pd.read_csv(csv_path3)
csv_path4 = '../PBL_result_4.csv'
csv_file4 = pd.read_csv(csv_path4)
csv_path5 = '../PBL_result_5.csv'
csv_file5 = pd.read_csv(csv_path5)

In [ ]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = csv.writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [ ]:
#initial csv file
if not Resave and os.path.exists(csv_file_path):
    print('This csv file is exist,if want to resave please modify "Resave" parameter.')
else:
    print('Initial the csv.')
    with open(csv_file_path, 'w', newline='\n') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Image_name', 'Tooth_num', 'LPBL(%)', 'L_Stage(int)', 'RPBL(%)', 'R_Stage(int)','Dis_L','Dis_R','Dis_root','points_info','tooth_pbl_line_path'])

In [ ]:
csv_file_list = [csv_file1, csv_file2, csv_file3, csv_file4, csv_file5]


for csv_file in tqdm(csv_file_list, total = len(csv_file_list)):
    for index, row in csv_file.iterrows():
        #排除 data pollution
        if row['Image_name'] in train_img_name_lis:
            print("train data skip")
            continue
        img_name = row['Image_name']
        fill_up_name = row['Tooth_num']
        LPBL = row['LPBL(%)']
        L_Stage = row['L_Stage(int)']
        RPBL = row['RPBL(%)']
        R_Stage = row['R_Stage(int)']
        Dis_L = row['Dis_L']
        Dis_R = row['Dis_R']
        Dis_root = row['Dis_root']
        points_info = row['points_info']
        tooth_pbl_line_path = row['tooth_pbl_line_path']
        row_contents = [img_name, fill_up_name, LPBL, L_Stage , RPBL, R_Stage,Dis_L,Dis_R,Dis_root,points_info,tooth_pbl_line_path]
        append_list_as_row(csv_file_path, row_contents)

In [ ]:
def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

def point_in_box(point_boxs_list,check_point):
    point_pair_list = [[point_boxs_list[i-1],point_boxs_list[i]] for i in range(len(point_boxs_list))]
    box_area = PolygonArea(point_boxs_list)
    count_area = 0
    for edge in point_pair_list:
        edge.append(check_point)
        count_area = PolygonArea(edge)+count_area
    if count_area - box_area < 1:
        return True
    else:
        return False
    
def show_pixel_set(img_nparray):
    a = img_nparray
    unique, counts = np.unique(a, return_counts=True)
    return dict(zip(unique, counts))

def get_check_point(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(np.where(img == 255,255.,0.).astype('uint8'), cv2.COLOR_BGR2GRAY)
    # plt.imshow(img)
    # plt.show()
    img = np.where(img == 76,255,0)
    # plt.imshow(img)
    # plt.show()
    return (np.mean(np.nonzero(img)[1]),np.mean(np.nonzero(img)[0]))

#print_ = False
def get_DM(num,side):
    '''
    tooth_num:
    if tooth_num <= 8 and tooth_num >= 25:
        L = D  
        R = M
    else:
        L = M
        R = D        
    '''
    if ((int(num) <= 8 or int(num) >= 25) and side =='L') or (25 > int(num) > 8 and side =='R'):
        return '{}D'.format(num)
    else:
        return '{}M'.format(num)
    
def replac_value(image_name,M_tooth_info_b,D_info):
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'PBL_predict_tooth'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'PBL_value'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'PBL_stage'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'tooth_path'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'CEJ_points'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'Bone_level_points'] = '{}'.format('No_match')
    for tooth in M_tooth_info_b['match_tooth']:
        if tooth != 'No_match':
            tooth_num = tooth.split('_')[0]
            tooth_side = tooth.split('_')[1]
            tooth_left = get_DM(tooth_num,'L') if 'L' in tooth_side else None
            tooth_right = get_DM(tooth_num,'R') if 'R' in tooth_side else None

            LPBL = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['LPBL(%)'].values[0]
            L_Stage = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['L_Stage(int)']
            RPBL = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['RPBL(%)'].values[0]
            R_Stage = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['R_Stage(int)']
            tooth_path = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['Tooth_num'].values[0]
            
            tooth_info = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['points_info'].values[0][1:-1]
            #print(tooth_info)
            tooth_info_list = tooth_info.split(',')
            L_points_info = tuple((tooth_info_list[0],tooth_info_list[1]))
            R_points_info = tuple((tooth_info_list[2],tooth_info_list[3]))
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_left)), 'PBL_predict_tooth'] = '{}'.format(tooth_num+'_L') 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_right)), 'PBL_predict_tooth'] = '{}'.format(tooth_num+'_R')
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_left)), 'PBL_value'] = '{}'.format(LPBL,'.3f') 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_right)), 'PBL_value'] = '{}'.format(RPBL,'.3f')
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_left)), 'PBL_stage'] = '{}'.format(L_Stage.values[0])
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_right)), 'PBL_stage'] = '{}'.format(R_Stage.values[0])
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_left)), 'tooth_path'] = '{}'.format(tooth_path) 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_right)), 'tooth_path'] = '{}'.format(tooth_path) 
            
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_left)), 'CEJ_points'] = '{}'.format(int(tooth_info_list[0])) 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_right)), 'CEJ_points'] = '{}'.format(int(tooth_info_list[2]))  
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_left)), 'Bone_level_points'] = '{}'.format(int(tooth_info_list[1])) 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_side'] == '{}'.format(tooth_right)), 'Bone_level_points'] = '{}'.format(int(tooth_info_list[3]))     
    return D_info


## Load model_detection csv & dentist_prediction 

In [ ]:
#Model_detection_result_csv_path = '/root/notebooks/tooth_project/tooth_xray_segmention/PBL_detection/csv_file/PBL_result_512.csv'
Model_detection_result_csv_path = csv_file_path

In [ ]:
Model_detection = pd.read_csv(Model_detection_result_csv_path)
Dentist_predict = pd.read_csv(Dentist_predict_result_csv_path)

## Show info check non_Null

In [ ]:
# Model_detection.info()
# print('------------------------------------------------')
# Dentist_predict.info()

## Show the first three data

In [ ]:
Model_detection.head(3)

In [ ]:
Dentist_predict.head(3)

In [ ]:
D_tooth_info = Dentist_predict
D_tooth_info["PBL_predict_tooth"] = np.nan
D_tooth_info["PBL_value"] = np.nan
D_tooth_info["PBL_stage"] = np.nan
D_tooth_info["tooth_path"] = np.nan
D_tooth_info["CEJ_points"] = np.nan
D_tooth_info["Bone_level_points"] = np.nan
D_tooth_info.head(3)

## Check comparsion image name

In [ ]:
# json_save_path = '../json_file/*.json'
# json_file_list = glob.glob(json_save_path)
# assert len(json_file_list) > 0, 'wrong json path'
# print(Model_detection.columns)
# Model_detection['tooth_side'] = ""
# for index, row in Model_detection[:5].iterrows():
#     img_path = row['tooth_pbl_line_path']
#     # check_path = ('.'+img_path).replace("3_keypoint_test/2_seg_img_with_cej","2_seg_img_with_cej")
#     check_path = ('.'+img_path)
#     check_point = get_check_point(check_path)
#     json_file_path = [json_name for json_name in json_file_list if row['Image_name'] in json_name]
#     if json_file_path != []:
#         with open(json_file_path[0], 'r') as file:
#             json_file = json.load(file)
#         for tooth in json_file['shapes']:
#             check_in_box = point_in_box(tooth['points'], check_point)
#             if check_in_box:   
#                 Model_detection.loc[index, 'tooth_side'] = tooth['label']
#                 break
#     else:
#         print(row['Image_name'], 'have no json file')
# print(Model_detection.head(5))

In [ ]:
list_image_name = list(set(list(Model_detection['Image_name'])))
list_image_name.sort()
# list_image_name = list_image_name[:2]

json_save_path = '../../data/json_file/*.json'
json_file_list = glob.glob(json_save_path)
assert len(json_file_list) > 0, 'wrong json path'


show = True
json_ = True

count = 0

for image_name in tqdm(list_image_name ,total=len(list_image_name)):
    match_tooth = []
    M_tooth_info = Model_detection.loc[(Model_detection['Image_name'] == image_name)]
    if True:
        for index, img_path in enumerate(M_tooth_info[['tooth_pbl_line_path'][0]]):
            check_in_box = False
            True_tooth_label = "No_match"
            if json_:
                #check_path = ('.'+img_path).replace("3_pattern_detection_result","2_seg_img_with_cej")
                check_path = ('.'+img_path).replace("3_keypoint_test/2_seg_img_with_cej","2_seg_img_with_cej")
                check_point = get_check_point(check_path)
                json_file_path = [json_name for json_name in json_file_list if image_name in json_name]
                if json_file_path != []:
                    with open(json_file_path[0], 'r') as file:
                        json_file = json.load(file)
                    for tooth in json_file['shapes']:
                        check_in_box = point_in_box(tooth['points'],check_point)
                        if check_in_box:   
                            True_tooth_label = tooth['label']
                            break
            count += 1
            match_tooth.append(True_tooth_label)
    tooth = match_tooth
    M_tooth_info['match_tooth'] = tooth
    D_tooth_info = replac_value(image_name, M_tooth_info, D_tooth_info)
print(count)

In [ ]:
print(len(json_file_list))

In [ ]:
D_tooth_info_b = D_tooth_info[['image_name','tooth_side','stage','PBL_predict_tooth','PBL_value','PBL_stage','tooth_path','CEJ_points','Bone_level_points']]
M_tooth_info_b = M_tooth_info[['Image_name','LPBL(%)','L_Stage(int)','RPBL(%)','R_Stage(int)']]

In [ ]:
D_tooth_info_b = pd.DataFrame(D_tooth_info_b)
D_tooth_info_b.to_csv(result_file_path)

In [ ]:
Dentist_predict_image_name = Dentist_predict["image_name"]

In [ ]:
Dentist_predict_image_name.head()

In [ ]:
# Test
df = pd.read_csv('../csv_file/PBL_match_result_fold4.csv')
print(df.columns)
print(len(df))
# Test A
assert len(df) == len(df.drop_duplicates())
# Test B
new_df = df.drop(columns=['Unnamed: 0'])
# new_df = df.drop(columns=['Unnamed: 0.1'])
new_df = new_df.drop_duplicates()
print(len(new_df))
assert len(df) == len(new_df)